In [1]:
import os
import pandas as pd
import numpy as np

In [8]:
icgc_mut = "../data/datasets/PCAWG/mutations/snv_mnv_indel/final_consensus_passonly.snv_mnv_indel.icgc.public.maf"
tcgc_mut = "../data/datasets/PCAWG/mutations/snv_mnv_indel/final_consensus_passonly.snv_mnv_indel.tcga.controlled.maf"
icgc_drivers = "../data/datasets/PCAWG/driver_mutations/TableS3_panorama_driver_mutations_ICGC_samples.public.tsv"
tcga_drivers = "../data/datasets/PCAWG/driver_mutations/TableS3_panorama_driver_mutations_TCGA_samples.controlled.tsv"
cnv_data = "../data/datasets/PCAWG/mutations/cnv"
suppl_table = "../data/datasets/PCAWG/supplementary Tables/Supplementary Table 1.csv"
mut_dir = "../data/snv_mv_indels_by_cancer_subtype"
GENOME_WIDE_MUT_DATA = "../data/genome_wide_mutation_data.tsv"

In [3]:
suppl_data = pd.read_csv(suppl_table, sep=",", header=0)
whitelisted_samples = suppl_data["tumour_specimen_aliquot_id"].unique().tolist()
print(f"Number of unique samples in the whitelist: {len(whitelisted_samples)}")
suppl_data

Number of unique samples in the whitelist: 2583


,tumour_specimen_aliquot_id,normal_specimen_aliquot_id,icgc_donor_id,icgc_sample_id,icgc_specimen_id,dcc_specimen_type,project_code,gender,age,organ_system,...,specimen_donor_treatment_type,donor_wgs_included_excluded,specimen_library_strategy,gain_count,loss_count,hd_count,wgd,all.SNVs,all.MNVs,all.Indels
0,0009b464-b376-4fbc-8a56-da538269a02f,5ef2ed4d-464e-4a51-95e0-401d9ae7be86,DO46416,SA505245,SP101724,Recurrent tumour - other,OV-AU,female,54.0,OVARY,...,other therapy,Included,WGS,288,349,0,1,15273,225,670
1,003819bc-c415-4e76-887c-931d60ed39e7,9a1c7950-69c9-4f27-800c-8bd834bc3d08,DO36062,SA413814,SP79365,Primary tumour - solid tissue,PBCA-DE,female,4.0,"BRAIN, & CRANIAL NERVES, & SPINAL CORD, (EXCL....",...,no treatment,Included,WGS,1,0,0,0,104,0,11
2,0040b1b6-b07a-4b6e-90ef-133523eaf412,64d942b9-eb38-4c98-8999-4b682377ee85,DO45049,SA501385,SP98853,Primary tumour - solid tissue,LINC-JP,male,73.0,LIVER,...,NaN,Included,WGS,40,19,1,0,14756,135,932
3,00493087-9d9d-40ca-86d5-936f1b951c93,d5dfac45-4aff-481d-9117-807ed47b7b10,DO22145,SA262459,SP47708,Primary tumour - solid tissue,LGG-US,female,47.0,"BRAIN, & CRANIAL NERVES, & SPINAL CORD, (EXCL....",...,NaN,Included,WGS,1,8,0,0,2536,1,241
4,00508f2b-36bf-44fc-b66b-97e1f3e40bfa,a3cc9015-6131-417c-8b71-e6028a4646bd,DO48578,SA514933,SP106808,Primary tumour - solid tissue,PAEN-AU,female,59.0,PANCREAS,...,no treatment,Included,WGS,3,65,2,0,2168,3,154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2578,ff870342-f0d6-4450-8f9c-344c046a0baf,8d6563a5-9777-4b2a-a4ac-00b59e6729ff,DO51079,SA530530,SP112845,Primary tumour - solid tissue,PRAD-CA,male,60.0,PROSTATE GLAND,...,no treatment,Included,WGS,5,32,4,0,2584,11,247
2579,ffa976f0-aa60-4867-842e-361afa7d68ac,36021aee-1b89-4d5d-94a8-6b9db44a4448,DO52704,SA542175,SP116740,Primary tumour - blood derived (peripheral blood),CLLE-ES,male,71.0,"BLOOD, BONE MARROW, & HEMATOPOIETIC SYS",...,no treatment,Included,WGS,0,2,0,0,3247,23,178
2580,ffad9288-c622-11e3-bf01-24c6515278c0,ff27bf50-c622-11e3-bf01-24c6515278c0,DO23552,SA270468,SP50185,Primary tumour - solid tissue,LIRI-JP,female,80.0,LIVER,...,no treatment,Included,WGS,43,73,0,0,7998,78,529
2581,ffdd4feb-aca3-4104-b1e8-954d705a6450,db3bb39b-a1a6-49af-aae3-7cb489a724e4,DO720,SA4138,SP1491,Primary tumour - solid tissue,BLCA-US,female,80.0,URINARY BLADDER,...,NaN,Included,WGS,12,50,1,1,12007,58,657


In [4]:
icgc_mut = pd.read_csv(icgc_mut, sep="\t", low_memory=False)
icgc_mut = icgc_mut = icgc_mut[["Hugo_Symbol", "Chromosome", "Start_position", "End_position", "Strand", "Reference_Allele", "Tumor_Seq_Allele1", "Tumor_Seq_Allele2", "Variant_Classification", "Tumor_Sample_Barcode", "Project_Code", "Donor_ID"]]
tcga_mut = pd.read_csv(tcgc_mut, sep="\t", low_memory=False)
tcga_mut = tcga_mut[["Hugo_Symbol", "Chromosome", "Start_position", "End_position", "Strand", "Reference_Allele", "Tumor_Seq_Allele1", "Tumor_Seq_Allele2", "Variant_Classification", "Tumor_Sample_Barcode", "Project_Code"]]
mut_df = pd.concat([icgc_mut, tcga_mut], axis=0, ignore_index=True)

In [5]:
drivers_icgc = pd.read_csv(icgc_drivers, sep="\t", low_memory=False)
drivers_tcga = pd.read_csv(tcga_drivers, sep="\t", low_memory=False)
driver_mut = pd.concat([drivers_icgc, drivers_tcga], axis=0, ignore_index=True)
driver_mut.head()

,sample_id,ttype,chr,pos,ref,alt,gene,driver,driver_statement,category,top_category,biallelic
0,03c3c692-8a86-4843-85ae-e045f0fa6f88,Panc-AdenoCA,x,x,x,x,TSC1,known,known,coding_tsg_breakpoint,SV,no
1,09497b9b-6fca-48cb-af97-161a3e434a51,Eso-AdenoCa,x,x,x,x,NOTCH1,known,known,coding_tsg_breakpoint,SV,no
2,0bfd1043-8170-e3e4-e050-11ac0c4860c5,Prost-AdenoCA,x,x,x,x,CBLB,known,known,coding_tsg_breakpoint,SV,no
3,0bfd1043-8183-e3e4-e050-11ac0c4860c5,Prost-AdenoCA,x,x,x,x,PTEN,known,known,coding_tsg_breakpoint,SV,no
4,0e872e0d-4711-4364-a5d0-6beeb6fd3ff2,Eso-AdenoCa,x,x,x,x,ASXL1,known,known,coding_tsg_breakpoint,SV,no


In [6]:
all_samples = mut_df[["Tumor_Sample_Barcode", "Project_Code"]].drop_duplicates()
all_samples["whitelisted"] = all_samples["Tumor_Sample_Barcode"].apply(lambda x: True if x in whitelisted_samples else False)
all_samples["tissue"] = all_samples["Project_Code"].apply(lambda x: x.split("-")[0])
all_samples.rename(columns={
    "Tumor_Sample_Barcode": "sample_id",
	"Project_Code": "ttype"
}, inplace=True)
all_samples = all_samples[all_samples["whitelisted"] == True]
print(f"Number of unique samples in the whitelist: {len(all_samples)}")
all_samples

Number of unique samples in the whitelist: 2583


,sample_id,ttype,whitelisted,tissue
0,0009b464-b376-4fbc-8a56-da538269a02f,Ovary-AdenoCA,True,Ovary
16168,003819bc-c415-4e76-887c-931d60ed39e7,CNS-PiloAstro,True,CNS
16283,0040b1b6-b07a-4b6e-90ef-133523eaf412,Liver-HCC,True,Liver
32106,00508f2b-36bf-44fc-b66b-97e1f3e40bfa,Panc-Endocrine,True,Panc
34431,005794f1-5a87-45b5-9811-83ddf6924568,Kidney-RCC,True,Kidney
...,...,...,...,...
52395435,fea5827b-251d-474b-8713-fc76db995fe7,Ovary-AdenoCA,True,Ovary
52404175,feccc27e-5c2e-4339-8746-0945fee93c1e,Head-SCC,True,Head
52407286,feccee20-a62d-4152-b832-b9fdaca87a61,Stomach-AdenoCA,True,Stomach
52419917,ff3fe4a3-7d19-4d4d-81b6-aaa41ba5bf39,Ovary-AdenoCA,True,Ovary


In [7]:
driver_count = driver_mut.groupby(["sample_id", "ttype"]).size().reset_index(name="num_drivers")
print(driver_count.shape)
driver_count = all_samples.merge(driver_count, how="left", on=["sample_id", "ttype"])
driver_count["num_drivers"] = driver_count["num_drivers"].fillna(0)
print(driver_count.shape)
driver_count

(2354, 3)
(2583, 5)


,sample_id,ttype,whitelisted,tissue,num_drivers
0,0009b464-b376-4fbc-8a56-da538269a02f,Ovary-AdenoCA,True,Ovary,13.0
1,003819bc-c415-4e76-887c-931d60ed39e7,CNS-PiloAstro,True,CNS,1.0
2,0040b1b6-b07a-4b6e-90ef-133523eaf412,Liver-HCC,True,Liver,6.0
3,00508f2b-36bf-44fc-b66b-97e1f3e40bfa,Panc-Endocrine,True,Panc,8.0
4,005794f1-5a87-45b5-9811-83ddf6924568,Kidney-RCC,True,Kidney,5.0
...,...,...,...,...,...
2578,fea5827b-251d-474b-8713-fc76db995fe7,Ovary-AdenoCA,True,Ovary,6.0
2579,feccc27e-5c2e-4339-8746-0945fee93c1e,Head-SCC,True,Head,0.0
2580,feccee20-a62d-4152-b832-b9fdaca87a61,Stomach-AdenoCA,True,Stomach,7.0
2581,ff3fe4a3-7d19-4d4d-81b6-aaa41ba5bf39,Ovary-AdenoCA,True,Ovary,9.0


In [9]:
# percentage of samples with driver mutations
driver_count["has_driver"] = driver_count["num_drivers"].apply(lambda x: 1 if x > 0 else 0)
driver_count["has_driver"] = driver_count["has_driver"].astype(int)
counts = driver_count["has_driver"].value_counts()
print(counts)
print(f"Percentage of samples with driver mutations: {counts[1] / counts.sum() * 100:.2f}%")
# average driver count per sample
mean_driver_count = driver_count["num_drivers"].mean()
print(f"Mean driver count: {mean_driver_count}")
# average driver count per tissue type
mean_driver_count_by_tissue = driver_count.groupby("tissue")["num_drivers"].mean().reset_index()
mean_driver_count_by_tissue.sort_values("num_drivers", ascending=False, inplace=True)
mean_driver_count_by_tissue

1    2354
0     229
Name: has_driver, dtype: int64
Percentage of samples with driver mutations: 91.13%
Mean driver count: 5.304297328687572


,tissue,num_drivers
1,Bladder,11.347826
20,Uterus,9.818182
6,ColoRect,9.538462
11,Lung,8.476190
14,Ovary,8.036364
3,Breast,7.597156
7,Eso,7.484536
18,Stomach,7.455882
8,Head,7.196429
15,Panc,6.255591


In [11]:
genome_wide_mut = pd.read_csv(GENOME_WIDE_MUT_DATA, sep="\t", low_memory=False)
# genome_wide_mut = pd.merge(genome_wide_mut, driver_count, how="left", on=["sample_id", "ttype"])
# genome_wide_mut.to_csv(GENOME_WIDE_MUT_DATA, sep="\t", index=False)
genome_wide_mut

,Tumor_Sample_Barcode,total_mutations,num_drivers,cancer_type,cna_burden
0,00493087-9d9d-40ca-86d5-936f1b951c93,2778,5,CNS-Oligo,17.324180
1,2aeaab9f-4459-4be8-91e7-e0746cbd671c,3388,4,CNS-Oligo,19.993935
2,3a81f733-b3a3-4578-b925-a663612bc92c,2489,2,CNS-Oligo,17.647446
3,3d477ac1-3bdf-4410-9594-1a3a1412c6ae,5801,3,CNS-Oligo,24.948595
4,4853fd17-7214-4f0c-984b-1be0346ca4ab,2004,2,CNS-Oligo,9.704638
...,...,...,...,...,...
2773,ec31e921-0782-4f77-ac23-4190af1e61ee,5524,4,Uterus-AdenoCA,32.833018
2774,ed32c725-08ae-48eb-8fa2-719b9aeb7550,16511,2,Uterus-AdenoCA,93.072360
2775,f7187888-f261-4a39-99b4-966fd7207117,30970,3,Uterus-AdenoCA,11.054963
2776,f7dcc2e4-1fc4-4b39-bc6a-720e66116d68,35812,7,Uterus-AdenoCA,89.617897
